In [39]:
import os
import subprocess
import pandas as pd
import numpy as np

In [40]:
#lists all the files in the directory

sequenc=[]
dicti_r={}
app=0
for nu in range(1,61):
    filenames_filt=[]
    filenames=os.listdir("/mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v4/rnarobo_results/"+str(nu))
    for fi in filenames:
        if fi[-3:]=="txt":
            filenames_filt.append(fi)

    for filename in filenames_filt:
       with open(os.path.join("/mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v4/rnarobo_results/"+str(nu), filename), 'r') as f: # open in readonly mode
          # do your stuff
            f_l=f.readlines()
            #means at least one ribo was found
            if len(f_l)>17:
                for linea in f_l[11:-6]:
                    #lets see if we want to append the sequence (means we found a ribo before)
                    if app==1:
                        sequenc.append(linea)
                        app=0
                    if linea[0]!="|":
                        #line=linea.replace(".","_")
                        line=linea.strip().split()
                        try:
                            if line[2] in dicti_r:
                                dicti_r[line[2]].append(int(line[0]))
                            else:
                                dicti_r[line[2]]=[int(line[0])]
                            #set the var to 1: add the next line to sequences
                            app=1 
                        except: 
                            pass
                    



In [6]:
allribos=pd.DataFrame(columns=["rnarobo_seq"])

In [7]:
allribos["rnarobo_seq"]=sequenc

In [8]:
#total number of all unique ribozymes in WGS databases
len(allribos["rnarobo_seq"].unique())

9344

In [9]:
df1str=allribos.to_csv(sep="\t",index=False)
f= open("tables_and_results/all_WGS_ribos.csv","w+")
f.write(df1str)
f.close()

In [10]:
#to create dicti of all trnascan files
dicti={}
for nu2 in range(1,61):
    filenames=os.listdir("/mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v4/trnascan/"+str(nu2))
    filenames_filt=[]
 
    for fi in filenames:
        if fi[-3:]=="txt":
            filenames_filt.append(fi)
    for filename in filenames_filt:
        #with open(os.path.join("/mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v3_T/trnascan/"+str(nu), filename), 'r') as f: # open in readonly mode
            f=open("/mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v4/trnascan/"+str(nu2)+"/"+filename, 'r')
           
          # do your stuff
            f_l=f.readlines()
         
            #means at least one ribo was found
            if len(f_l)>1:
                for line in f_l[3:]:
                #create a dictionary with seq name as key and the starting position as value
                    line=line.strip().split()
                    #line[0]=line[0].replace(".","_")
                    if line[0] in dicti:
                        dicti[line[0]].append(int(line[3]))
                    else:
                        dicti[line[0]]=[int(line[3])]
                    

In [11]:
matching=[]
for key in dicti_r:
    if key in dicti:
        for value in dicti_r[key]:
            for value2 in dicti[key]:
                if value-value2 in range(-5,6):
                    matching.append([key,value,value2])

In [14]:
newm=matching

In [ ]:
#lets create the dataframe with all hits

In [16]:
lets_do_this=pd.DataFrame(columns=["ID","trna_nr","trna_start","trna_end","trna_type","codon","ins_start","ins_end","trna_confidence","psuedo","rnarobo_start","rnarobo_end","rnarobo_seq"])
check=0
oldn=""
duplicates=[]
for nr in range(len(newm)):
    #check: if old filename is still the same
    match=newm[nr]
    lin_df=""
    if match[0]!=oldn:
        
        
        trnaf=open("/mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v4/together/"+match[0].split(".")[0]+"trna_result.txt")
        trna=trnaf.readlines()
        trnaf.close()
        
        rnarobof=open("/mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v4/together/"+match[0].split(".")[0]+"_rnarobo_result.txt")
        rnarobo=rnarobof.readlines()
        rnarobof.close()
        
        
        
        
    for line in trna[3:]:
        
    #create a dictionary with seq name as key and the starting position as value
        line=line.strip().split()
        
        if match[0]==line[0]:
            if len(lin_df.split())==0:
                if str(match[2])==line[3]:
                    #print(line)
                    lin_df+=" ".join(line)+" "
                    if lin_df[-2]!="o":
                        lin_df+="NA "
            else: 
                duplicates.append(line)    
    
    for line in rnarobo[11:-6]:
        #fix 1: we don't need this here!
        #line=line.replace(".","_")
        line=line.strip().split()
        if len(lin_df.split())<13:

            if check==1:
                #if len(lin_df.split())<13:
                    lin_df+=str(line[0])
                    check=0
                #else:
                        #duplicates.append(line)

                #check=0
                
            #####FIX 2: Important here: we did this to make sure that we are actually matching the right ribozyme here...
            if len(line)>1:
                #here we made the mistake: not checking the actual ID but only starting position
                if match[0]==line[2]:
                    if str(match[1])==line[0]:
                    #if len(lin_df.split())<13:
                        lin_df+=" ".join(line[:2])
                        lin_df+=" "
                        check=1
                #else:
                #duplicates.append(line)
                #check=0
    
    #change to 4: CORRECT
   
    
    oldn=match[0]
    lin_f=lin_df.split()
    lets_do_this.loc[nr]=np.array(lin_f)
    
    #lets_do_this.iloc[nr,:8]=lin_df[0]
    #lets_do_this.iloc[nr,8:-1]=lin_df[1]
    #lets_do_this.iloc[nr,-1]=lin_df[2]

In [17]:
fff=lets_do_this

In [19]:
y=fff["trna_type"].value_counts()

In [22]:
fff["trnaseq"]= " "

fff["ribozyme_control"]= " "

In [ ]:
# add the trna sequences
#now we have the problem that we have different formats in the files: some have only one line for seuence, others multiple

In [24]:
#lets try to save and cut out the respective trnas (and in between part)
check=0
for ID in range(len(fff)):
    match=fff.iloc[ID,0]
    rnarobof=open("/mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v4/together/"+match.split(".")[0]+"_seqs_rnarobo_result.txt")
    rnarobo=rnarobof.readlines()
    rnarobof.close()
    #inp="grep -A1 -m1 "+fff.iloc[ID,0]+" mgv_contigs.fna" 
    #res=subprocess.run(inp,shell=True, capture_output=True, text=True)
    sequ=""
    for num in rnarobo:
        
        if num[0]==">":
            check=0
        #in this order it automatecely selects the sequence after matching
        if check==1:
            sequ+=num.strip()
            #check=0
            #exclude the ">" of the fasta file
        if match==num.split(" ")[0][1:]:
            check=1

        #sequ=res.stdout.split()[-1]
    check=0
    start=int(fff.iloc[ID,2])-1
    end=int(fff.iloc[ID,3])
    start2=int(fff.iloc[ID,10])-1
    end2=int(fff.iloc[ID,11])
    if start < end:
        trna=sequ[start:end]
        rnarobo=sequ[start2:end2]
        fff.iloc[ID,-2]=trna
        fff.iloc[ID,-1]=rnarobo
    if end < start:
        #reverse sequence: add +1 to reverse previous indexing correction 
        end-=1
        start+=1
        end2-=1
        start2+=1
        trna_rev=sequ[end:start]
        ribo_rev=sequ[end2:start2]
        trna=[]
        rnarobo=[]
        #reverse parts: transcribe back into the correct format!
        for nuc in range(len(trna_rev)-1,-1,-1):
            if trna_rev[nuc]=="A":
                trna.append("T")
            if trna_rev[nuc]=="T":
                trna.append("A")
            ####here we also added the N option, qhich is superrare but possible nontheless
            if trna_rev[nuc]=="N":
                trna.append("N")
            if trna_rev[nuc]=="G":
                trna.append("C")
            if trna_rev[nuc]=="C":
                trna.append("G")
        for nuc in range(len(ribo_rev)-1,-1,-1):
            if ribo_rev[nuc]=="A":
                rnarobo.append("T")
            if ribo_rev[nuc]=="T":
                rnarobo.append("A")
            if ribo_rev[nuc]=="G":
                rnarobo.append("C")
            if ribo_rev[nuc]=="C":
                rnarobo.append("G")
            if ribo_rev[nuc]=="N":
                rnarobo.append("N")
        trna="".join(trna)
        rnarobo="".join(rnarobo)
        fff.iloc[ID,-2]=trna
        fff.iloc[ID,-1]=rnarobo
    

In [25]:
#check if the control and actual seq are the same
for x in range(len(fff)):
    seq1=fff.loc[x,"rnarobo_seq"]
    seq3="".join(seq1.split("|"))
    seq2=fff.loc[x,"ribozyme_control"]
    if seq2!=seq3:
        print("something wrong",x)
        print(seq2)
        print(seq3)

something wrong 9784

GGTGCCATGAGAGAAGCCTGCACGCGGCGGCACTGCTTTAACTGATTCTCTC


In [27]:
#wrong way: false positive, lets remove it
fff.iloc[9783:9785,:]

,ID,trna_nr,trna_start,trna_end,trna_type,codon,ins_start,ins_end,trna_confidence,psuedo,rnarobo_start,rnarobo_end,rnarobo_seq,trnaseq,ribozyme_control
9783,ERR2597408.26235469,1,19,90,Met,CAT,0,0,64.9,NA,91,145,|GGGATAC|AT|TTGATAA|GCC|T|CC|ACGT|GGC|GTATCCT|...,TGCGGGGTGTAGCAGTGGTAGCTTACCAGCCTCATAAGCTGGTGGT...,GGGATACATTTGATAAGCCTCCACGTGGCGTATCCTGGGTAATGCT...
9784,SRR7141503.13486898,1,196,100,Pro,TGG,0,0,19.5,pseudo,98,149,|GGTGCC|AT|GAGAGAA|GCC|T|GC|ACGC|GGC|GGCACT|GC...,TAGCGCAGCTGGTAGAGCGCGTGATTTGGGATCACGAGGTCGTGAG...,


In [28]:
ft=fff.copy(deep=True)

In [29]:
#drop the ribo with trna going into the wrong direction
ft.drop(9784,inplace=True)

In [31]:
ft.reset_index(drop=True,inplace=True)

In [32]:
df1str=ft.to_csv(sep="\t",index=False)
f= open("tables_and_results/WGS_ribo_trna.csv","w+")
f.write(df1str)
f.close()

In [42]:
#unique ribozymes in metag samples
len(ft["ribozyme_control"].unique())

1698

In [44]:
#unique trnas in metag samples
len(ft["trnaseq"].unique())

5721

In [ ]:
#check the environments in which the hits are found